In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import requests
import bs4
import json
from requests.auth import HTTPBasicAuth
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-d2088fc5-5f70-4703-8360-03eabf53dcca"

In [3]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [4]:
print(sumids[0])

xVfAvKB6MsfZ-ihvaeOpFFjCSWq8e9ba07jtBe4E0IYw5mY


In [5]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [6]:
print(accids[0])

0LKuAA-YP37tFAKtCSLfvyH8qSirbyYxXQeQH2HlIj0xWMg


In [8]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [9]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

20500
11127


In [10]:
with open('temp.txt', 'w') as f:
    for item in gamids:
        f.write("%s\n" % item)

In [28]:
with open('euw_chall_gam_id_list1.txt', 'r') as f:
    finchallgames = f.read().splitlines()

In [29]:
finchallgames =set([int(e) for e in finchallgames])

In [30]:
len(gamids - (finchallgames & gamids))

542

In [31]:
gamids = gamids - finchallgames

In [32]:
len(gamids)

542

In [34]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids:
    gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [22]:
gamreq.status_code

200

In [35]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(542, 1550, 1230)

In [36]:
# create df from matches

df = pd.DataFrame(matchlist)

In [41]:
df1 = pd.read_csv('euchall1.csv', low_memory=False)
df1

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_6_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_30-end,participants_7_timeline_csDiffPerMinDeltas_20-30,participants_7_timeline_xpDiffPerMinDeltas_30-end,participants_7_timeline_xpDiffPerMinDeltas_20-30,participants_7_timeline_damageTakenDiffPerMinDeltas_30-end,participants_7_timeline_damageTakenDiffPerMinDeltas_20-30,participants_8_timeline_csDiffPerMinDeltas_30-end,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end
0,4681072641,EUW1,1593268978835,1299,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4707844099,EUW1,1594758284675,1438,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4679991311,EUW1,1593201304458,1826,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4648730648,EUW1,1591566971144,1523,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4632641560,EUW1,1590840449379,1908,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10580,4655053348,EUW1,1591893108042,2045,420,11,13,10.12.324.5925,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10581,4712561190,EUW1,1595018522973,2000,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,-0.6,NaN,53.2,NaN,319.0,NaN,NaN,NaN
10582,4634278439,EUW1,1590926921649,1624,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10583,4703222319,EUW1,1594500857104,1275,450,12,13,10.14.327.8505,ARAM,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end,participants_9_timeline_creepsPerMinDeltas_30-end,participants_9_timeline_xpPerMinDeltas_30-end,participants_9_timeline_goldPerMinDeltas_30-end,participants_9_timeline_csDiffPerMinDeltas_30-end,participants_9_timeline_xpDiffPerMinDeltas_30-end,participants_9_timeline_damageTakenPerMinDeltas_30-end,participants_9_timeline_damageTakenDiffPerMinDeltas_30-end
0,4706728496,EUW1,1594710991563,1407,450,12,13,10.14.327.8505,ARAM,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4673501747,EUW1,1592847756780,1549,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4687067702,EUW1,1593606004783,2024,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4692015670,EUW1,1593886827577,1399,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4720228926,EUW1,1595431428869,1148,1300,21,13,10.15.328.8759,NEXUSBLITZ,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,4706992121,EUW1,1594728836373,1783,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,4704010235,EUW1,1594563154432,1225,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,4696047612,EUW1,1594135592032,1166,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,4688871421,EUW1,1593699385332,1248,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df = df1.append(df)

In [44]:
challgames = list(df['gameId'])
with open('euw_chall_gam_id_list.txt', 'w') as f:
    for item in challgames:
        f.write("%s\n" % item)

In [45]:
compression_opts = dict(method='zip',
                        archive_name='euchall.csv')  
df.to_csv('euchall.zip', index=False,
          compression=compression_opts)